In [2]:
# -*- coding:utf-8 -*-
import argparse
import warnings

warnings.filterwarnings('ignore')

import pickle, random, sys

sys.path.append('D:\\zhaixu\\Thesis_Code')

import os, pickle, random, sys

import numpy as np
import copy
import tensorflow.keras.backend as K

from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import model_from_json

from sklearn.model_selection import train_test_split

from mltools import build_model
from mltools import get_seq_data

# from rmlmodel.Sequence.CLDNN import CLDNNLikeModel
# from rmlmodel.Sequence.ResNet import ResNetLikeModel
# from rmlmodel.Sequence.VGG import VGGLikeModel

from rmlmodel.Sequence.vtcnn2 import VTCNN2
from rmlmodel.Sequence.CNN2 import CNN2
from rmlmodel.Sequence.CNN2Model import CNN2Model

from rmlmodel.Sequence.CLDNNLikeModel import CLDNNLikeModel
from rmlmodel.Sequence.CLDNNLikeModel1 import CLDNNLikeModel1
from rmlmodel.Sequence.CLDNNLikeModel2 import CLDNNLikeModel2

from rmlmodel.Sequence.CGDNN import CGDNN
from rmlmodel.Sequence.CuDNNLSTMModel import LSTMModel
from rmlmodel.Sequence.DAE import DAE
from rmlmodel.Sequence.DCNNPF import DCNNPF
from rmlmodel.Sequence.DenseNet import DenseNet
from rmlmodel.Sequence.GRUModel import GRUModel
from rmlmodel.Sequence.ICAMC import ICAMC
from rmlmodel.Sequence.MCLDNN import MCLDNN
from rmlmodel.Sequence.MCNET import MCNET
from rmlmodel.Sequence.PETCGDNN import PETCGDNN
from rmlmodel.Sequence.ResNet import ResNet

from trigger_config import load_data
from trigger_config import set_trigger_config

from plot_tools import plot_signal
from plot_tools import plot_constellation

In [98]:
X_train,X_test,Y_train,Y_test,mods,lbl,snrs,train_idx,test_idx = load_data()

筛选后的数据集总数： 110000
信噪比范围: 0 到 18
调制方式 11 种: ['8PSK', 'AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM']


In [95]:
def eval(model,X_test,Y_test):
    # model0:原模型测试
    classes = mods
    acc = {}
    for snr in snrs:

        # extract classes @ SNR
        test_SNRs = list(map(lambda x: lbl[x][1], test_idx))

        # CA
    #     test_X_i = X_test_benign.values[np.where(np.array(test_SNRs)==snr)]
    #     test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]   

        #ASR
        test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
        test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]   

        test_Y_i_hat = model.predict(test_X_i)
        # estimate classes
        test_Y_i_hat = model.predict(test_X_i)
        conf = np.zeros([len(classes),len(classes)])
        confnorm = np.zeros([len(classes),len(classes)])
        for i in range(0,test_X_i.shape[0]):
            j = list(test_Y_i[i,:]).index(1)
            k = int(np.argmax(test_Y_i_hat[i,:]))
            conf[j,k] = conf[j,k] + 1
        for i in range(0,len(classes)):
            confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
        #plt.figure()
        #plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))

        cor = np.sum(np.diag(conf))
        ncor = np.sum(conf) - cor
        #print("Overall Accuracy: ", cor / (cor+ncor))
        print(cor / (cor+ncor))
        acc[snr] = 1.0*cor/(cor+ncor)

In [71]:
X_train_modified, Y_train_modified = set_trigger_config(X_train.copy(), Y_train.copy(), pos_rate=0.1,
                                                        trigger_type='badnet', data_type='train')

# REP = IQ/AP/FFT
X_train_modified = get_seq_data(X_train_modified, seq_dtype ='IQ')

Poisoned training data.


In [72]:
X_test_modified, Y_test_modified = set_trigger_config(X_test.copy(), Y_test.copy(), pos_rate=0.1,
                                                        trigger_type='badnet', data_type='test')

# REP = IQ/AP/FFT
X_test_modified = get_seq_data(X_test_modified, seq_dtype ='IQ')

Evaluate ASR on poisoned test data.


In [99]:
X_test_benign = X_test.copy()

In [119]:
def complex_accumulate_features(x):
    
    # 转为复数形式
    x = x[:, 0, :] + 1j * x[:, 1, :]
    
    # 计算共轭
    x_r = np.conjugate(x)

    # 计算各种高阶统计量和累积量
    M20 = np.mean(np.abs(x)**2, axis=1, keepdims=True)
    M21 = np.mean(np.abs(x)**2, axis=1, keepdims=True)
    M22 = np.mean(np.abs(x_r)**2, axis=1, keepdims=True)
    M40 = np.mean(np.abs(x)**4, axis=1, keepdims=True)
    M41 = np.mean(np.abs(x)**2 * np.abs(x)**2, axis=1, keepdims=True)
    M42 = np.mean(np.abs(x)**4, axis=1, keepdims=True)
    M43 = np.mean(np.abs(x)**2 * np.abs(x_r)**2, axis=1, keepdims=True)
    M60 = np.mean(np.abs(x)**6, axis=1, keepdims=True)
    M63 = np.mean(np.abs(x)**6, axis=1, keepdims=True)
    M80 = np.mean(np.abs(x)**8, axis=1, keepdims=True)

    C20 = M20
    C21 = M21
    C40 = M40 - 3 * np.power(M20, 2)
    C41 = M41 - 3 * M20 * M21
    C42 = M42 - np.power(np.abs(M20), 2) - 2 * np.power(M21, 2)
    C60 = M60 - 15 * M20 * M40 + 30 * np.power(M20, 3)
    C63 = M63 - 6 * M41 * M20 - 9 * M42 * M21 + 18 * M20 * M20 * M21 + 12 * np.power(M21, 3)
    C80 = M80 - 28 * M20 * M60 - 35 * M40 * M40 + 420 * M20 * M20 * M40 - 630 * np.power(M20, 4)

    # 计算其他特征
    T1 = np.abs(C80) / np.power(np.abs(C42), 2)
    T2 = np.abs(C80) / np.power(np.abs(C40), 2)
    M1 = np.abs(C40) / np.abs(C42)
    M2 = np.power(np.abs(C63), 2) / np.power(np.abs(C42), 3)

    C20_norm = np.abs(C20 / C21)
    C40_norm = np.abs(C40 / np.power(C21, 2))
    C41_norm = np.abs(C41 / np.power(C21, 2))
    C42_norm = np.abs(C42 / np.power(C21, 2))
    C60_norm = np.abs(C60 / np.power(C21, 3))
    C63_norm = np.abs(C63 / np.power(C21, 3))
    C80_norm = np.abs(C80 / np.power(C21, 4))
    
    HOC = np.concatenate([M20,M21,M22,M40,M41,M42,M43,M60,M63,M80,
                   C20,C21,C40,C41,C42,C60,C63,C80],axis=1)
    
   # HOC = np.concatenate([C20_norm, C40_norm, C41_norm, C42_norm, C60_norm, C63_norm, C80_norm],axis=1)

    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    HOC = scaler.fit_transform(HOC)

    return HOC

In [127]:
from utlits import accumulat_features
X_train_complex = complex_accumulate_features(X_train)
X_train_modified_complex = complex_accumulate_features(X_train_modified)
X_test_modified_complex = complex_accumulate_features(X_test_modified)
X_test_benign_complex = complex_accumulate_features(X_test_benign)

In [138]:
np.concatenate([X_test_benign_complex,X_test_benign_complex],axis=1).shape

(11000, 36)

In [121]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_complex)
X_test_scaled = scaler.transform(X_test_benign_complex)

In [129]:
# from sklearn import datasets,model_selection,metrics,tree,preprocessing
# model=tree.DecisionTreeClassifier(
#     max_depth=35
# )
# # 模型训练
# model.fit(X_train_complex,Y_train)


import xgboost as xgb
from sklearn.multiclass import OneVsRestClassifier

model_xgb = xgb.XGBClassifier(max_depth=20, tree_method='gpu_hist', gpu_id=0)
model = OneVsRestClassifier(model_xgb)
model.fit(X_train_complex, Y_train)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=0, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=20,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, reg_alpha=None,
                                           

In [131]:
# 模型预测
y_pred=model.predict(X_test_benign_complex)
# 模型性能评价
print(metrics.accuracy_score(y_pred,Y_test))

# # 模型预测
# y_pred=model.predict(X_test_modified_complex)
# # 模型性能评价
# print(metrics.accuracy_score(y_pred,Y_test_modified))

0.3960909090909091


In [133]:
eval(model,X_test_benign_complex,Y_test)

0.22402287893231648
0.2940074906367041
0.345374449339207
0.39302112029384756
0.45281385281385284
0.4537205081669691
0.47069431920649235
0.4958828911253431
0.554016620498615
0.567591763652641


In [132]:
eval(model,X_test_modified_complex,Y_test_modified)

0.04671115347950429
0.05711610486891386
0.04757709251101322
0.050505050505050504
0.05194805194805195
0.06442831215970962
0.06402164111812443
0.05763952424519671
0.06001846722068329
0.05640107430617726
